## <font style="color: rgb(0,0,0);">Tutorial: </font> <font style="color: rgb(190,138,67);">Integrating STK with Python</font>

***

### STK Python API
The new STK Python API is only available with STK 12.1 or newer. If not installed, use pip to install it. Example code: pip install "C:\Program Files\AGI\STK 12\bin\AgPythonAPI\agi.stk<..ver..>-py3-none-any.whl". If using an older version of STK then use win32api or Comtypes.

### IntelliSense
When connected to STK via Python, while creating your variable, using the Tab key after periods enables IntelliSense which displays all of the options available off of the current interface. In the next section you will start STK 



In [ ]:
# uncomment the below to install the API, remember to modify your version, for example agi.stk12-12.2.0-py3-none-any.whl
#pip install "C:/Program Files/AGI/STK 12/bin/AgPythonAPI/agi.stk12<..ver..>-py3-none-any.whl"

### Problem Statement
You will be analyzing the behavior of a satellite when it has contact with a ground site. The task will be repetitive and you consider methods of automating the process and extracting the data. Knowing that you can integrate STK with other tools you decide to explore the process.

### Solution
Analysis in STK can be integrated and automated with code. You decide to run the process with Python. Using the resources on the STK Help and Github you explore how to model a mission with a script. 
From this tutorial you will learn how to:
- Connect STK to a programming interface
- Build a mission through a script
- Extract data from STK

### Set up your workspace

In [ ]:
# STK library imports
from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.stkobjects import *
from agi.stk12.stkutil import *
from agi.stk12.vgt import *
# if using astrogator uncomment the below
# from agi.stk12.stkobjects.astrogator
# if using aviator uncomment the below
# from agi.stk12.stkobjects.aviator

# Python helper library imports
import os

### Connect to the STK instance

In [ ]:
# Get reference to the current instance of STK
STK_PID = os.getenv('STK_PID')
stk = STKDesktop.AttachToApplication(pid=int(STK_PID))

# Grab a handle on the STK application root.
root = stk.Root 

### STK Root

Recall that the AgStkObjectRoot object is at the apex of the STK Object Model. The associated IAgStkObjectRoot interface will provide the methods and properties to load or create new scenarios and access the Object Model Unit preferences. Through the stk command you have a pointer to the IAgUiApplication interface; however the STK Python API provides a direct handle to the IAgStkObjectRoot via the Root property in STKDesktop or the NewObjectRoot() method in STKEngine.


Check that the root object has been built correctly, check the type()

In [ ]:
type(root)

# output will be 
# agi.stk12.stkobjects.AgStkObjectRoot

### Connect and design a new scenario

Now that you have launched STK via the Python interface, let's see if we can create a new scenario and set the time period via Python. We will create a new scenario, analysis period and reset the animation time.


In [ ]:
# 1. Define a scenario object

scenario = root.CurrentScenario

# 2. Set the analytical time period.

scenario.SetTimePeriod('Today','+24hr')

# 3. Reset the animation time to the newly established start time.

root.Rewind()

### Insert and configure objects

With a new scenario created, it's time to populate the scenario with objects. Use the STK Python API and the STK Connect commands, via the ExecuteCommand method,  to create a facility and a LEO satellite.

In [ ]:
# 1. Add a target object to the scenario. 
target  = AgTarget(scenario.Children.New(AgESTKObjectType.eTarget,"GroundTarget"))

# 2. Move the Target object to a desired location.

target.Position.AssignGeodetic(50,-100,0)

# 3. Add a Satellite object to the scenario.

satellite  = AgSatellite(root.CurrentScenario.Children.New(AgESTKObjectType.eSatellite,"LeoSat"))


Examine the below connect command before running. In it we will be using the Set State Classical connect command. Rather than manually setting the times we will use the define scenario times. 

Print them to confirm.

In [ ]:
print(scenario.StartTime)
print(scenario.StopTime)

In [ ]:
# 4. Propagate the Satellite object's orbit.

root.ExecuteCommand('SetState */Satellite/LeoSat Classical TwoBody "' + str(scenario.StartTime) + '" "' + str(scenario.StopTime) + '" 60 ICRF  "' + str(scenario.StartTime) + '" 7200000.0 0.0 90 0.0 0.0 0.0')

### Compute Access between objects

You now have a scenario with a Target object and a Satellite object. Determine when the Satellite object can access the Target object.

Locate the code needed to compute an access between two STK Objects using the IAgStkObject interface. The access is between the Satellite object and the Target Object.

HINT: Review the Python Code snippits
http://help.agi.com/stkdevkit/index.htm#stkObjects/ObjModPythonCodeSamples.htm



In [ ]:
access = satellite.GetAccessToObject(target)
access.ComputeAccess()

### Retrieve Access data from STK

Now that the scenario is fully built, the final task is to extract data and perform a basic analysis. We have just computed access between our two objects, we can use the STK data providers to pull data out of our scenario.

In [ ]:
# Retrieve and view the access data

accessDP         = access.DataProviders.Item('Access Data')

results          = accessDP.Exec(scenario.StartTime, scenario.StopTime)

accessStartTimes = results.DataSets.GetDataSetByName('Start Time').GetValues()

accessStopTimes  = results.DataSets.GetDataSetByName('Stop Time').GetValues()

print(accessStartTimes,accessStopTimes)

### Note: 

Generating the Start & Stop times in Python can also be pulled using the following lines of code.

More information available in [STK Object Model Tutorial](http://help.agi.com/stkdevkit/index.htm#stkObjects/ObjectModelTutorial.html). 

In [ ]:
accessIntervals = access.ComputedAccessIntervalTimes

dataProviderElements = ['Start Time', 'Stop Time']

for i in range(0,accessIntervals.Count):
    times = accessIntervals.GetInterval(i)
    print(times)

### Retrieve the Satellite Altitude Data From STK

Retrieve and view the altitude of the satellite during an access interval. In the following lines, note how the data providers must follow the data provider folder, sub-folder, and selection.

In [ ]:
satelliteDP       = satellite.DataProviders.Item('LLA State')

satelliteDP2      = satelliteDP.Group.Item('Fixed')

rptElements       = ['Time', 'Lat', 'Lon', 'Alt']

satelliteDPTimeVar = satelliteDP2.ExecElements(accessStartTimes,accessStopTimes, 60, rptElements)

satelliteAltitude = satelliteDPTimeVar.DataSets.GetDataSetByName('Alt').GetValues()

print(satelliteAltitude)

### Mission Complete 

You have just completed the STK integration with Python tutorial using Jupyter Notebooks. You have connected to the interface, built the mission, and extracted data. Don't forget to save your work. With your scenario safely saved, you can close out of STK. Workflows like these can be expanded and automated for quickly building and analyzing missions.
